#NOTES

Provided is the script used to to generate a baseline modle for comparision with an opitmized model. 
The baseline model is based off the xView2 baseline availiable at: https://github.com/DIUx-xView/xView2_baseline/tree/768b95e93ae56936e205bef0444137269a719755 though heavily modified.
It should be noted that as this research is not concerned with building detection and rather building classification, the locaiztion model has not been run however is functioning and instructions for use provided.In later secions, work arounds are provided if the locaisation model is not run.

It should be noted that this code is very sensitve and requires the file directory to be set up in a specific way. For this reason, arguments when executing code must be changed if running locally.

Finally, it is recommended that due to the size of the datasets and complexity of the model that this code is ran using GPU acceleration through google colab+ or google colab pro+ paid with a google drive of sufficient storage.

The second model provided as part of this research will be of the same format in a seperate document to avoid issues and confusion.


#Auxiliary
Used if cannot be run locally.

In [ ]:
%%writefile compute_mean.py

import argparse
import sys
import os

import six
import numpy as np

try:
	from PIL import Image
	available = True
except ImportError as e:
	available = False
	_import_error = e

from chainer.dataset import dataset_mixin


def _check_pillow_availability():
	if not available:
		raise ImportError('PIL cannot be loaded. Install Pillow!\n'
						  'The actual import error is as follows:\n' +
						  str(_import_error))


def _read_image_as_array(path, dtype):
	f = Image.open(path)
	try:
		image = np.asarray(f, dtype=dtype)
	finally:
		# Only pillow >= 3.0 has 'close' method
		if hasattr(f, 'close'):
			f.close()
	return image


class ImageDataset(dataset_mixin.DatasetMixin):
	
	def __init__(self, paths, root='.', dtype=np.float32):
		_check_pillow_availability()
		if isinstance(paths, six.string_types):
			with open(paths) as paths_file:
				paths = [path.rstrip() for path in paths_file]
		self._paths = paths
		self._root = root
		self._dtype = dtype

	def __len__(self):
		return len(self._paths)

	def get_example(self, i):
		path = os.path.join(self._root, self._paths[i])
		image = _read_image_as_array(path, self._dtype)

		if image.ndim == 2:
			# image is greyscale
			image = image[:, :, np.newaxis]
		return image.transpose(2, 0, 1)


def compute_mean(dataset):
	print('compute mean image')
	sum_color = 0
	N = len(dataset)
	for i, image in enumerate(dataset):
		sum_color += image.mean(axis=2, keepdims=False).mean(axis=1, keepdims=False)
		sys.stderr.write('{} / {}\r'.format(i, N))
		sys.stderr.flush()
	sys.stderr.write('\n')
	return sum_color / N


if __name__ == '__main__':

	parser = argparse.ArgumentParser(description='Compute images mean array')
	
	parser.add_argument('dataset',
						help='Path to training image-label list file')
	parser.add_argument('--root', '-R', default='.',
						help='Root directory path of image files')
	parser.add_argument('--output', '-o', default='mean.npy',
						help='path to output mean array')
	args = parser.parse_args()

	dataset = ImageDataset(args.dataset, args.root)
	mean = compute_mean(dataset)

	np.save(args.output, mean)

Writing compute_mean.py


In [ ]:
!python3 compute_mean.py "/content/drive/MyDrive/spacenet_gt/dataSet/train.txt" --root "/content/drive/MyDrive/Model_data/xBD/mexico-earthquake/images" --output "/content/drive/MyDrive/xView2_baseline-master/weights"

compute mean image
89 / 90


In [ ]:
%%writefile split_into_disasters.py
from os import walk, path, makedirs
from shutil import copy2 as cp


def get_files(base_dir):
    # Minmizing (halfing) list to just pre image files
    base_dir = path.join(base_dir, "images")
    files = [f for f in next(walk(base_dir))[2] if "pre" in f]

    return files


def move_files(files, base_dir, output_dir):
    for filename in files:
        disaster = filename.split("_")[0]

        # If the output directory and disater name do not exist make the directory
        if not path.isdir(path.join(output_dir, disaster)):
            makedirs(path.join(output_dir, disaster))

        # Check if the images directory exists
        if not path.isdir(path.join(output_dir, disaster, "images")):
            # If not create it
            makedirs(path.join(output_dir, disaster, "images"))

        # Move the pre and post image to the images directory under the disaster name
        cp(
            path.join(base_dir, "images", filename),
            path.join(output_dir, disaster, "images", filename),
        )
        post_file = filename.replace("_pre_", "_post_")
        cp(
            path.join(base_dir, "images", post_file),
            path.join(output_dir, disaster, "images", post_file),
        )

        # Check if the label directory exists
        if not path.isdir(path.join(output_dir, disaster, "labels")):
            # If not create it
            makedirs(path.join(output_dir, disaster, "labels"))

        pre_label_file = filename.replace("png", "json")
        # Move the pre and post label files to the labels directory under the disaster name
        cp(
            path.join(base_dir, "labels", pre_label_file),
            path.join(output_dir, disaster, "labels", pre_label_file),
        )
        post_label_file = pre_label_file.replace("_pre_", "_post_")
        cp(
            path.join(base_dir, "labels", post_label_file),
            path.join(output_dir, disaster, "labels", post_label_file),
        )


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description="split_into_disasters.py: Splits files under a single directory (with images/ and labels/ into directory of disasters/images|labels for the base submission pipeline (copies files)"
    )
    parser.add_argument(
        "--input",
        required=True,
        metavar="/path/to/dataset/train",
        help="Full path to the train (or any other directory) with /images and /labels",
    )
    parser.add_argument(
        "--output",
        required=True,
        metavar="/path/to/output/xBD",
        help="Full path to the output root dataset directory, will create disaster/images|labels under this directory",
    )

    args = parser.parse_args()

    files = get_files(args.input)
    move_files(files, args.input, args.output)

# Mount Drive and Check for GPU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(0, "content/drive/MyDrive/Colab Notebooks")

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2042219924683720063
 xla_global_id: -1]

#Localisation Spacenet Model

In [ ]:
%%writefile localisation_model.py

from __future__ import print_function

import argparse
import numpy as np

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

from unet import UNet
from dataset import LabeledImageDataset

from tensorboardX import SummaryWriter

import os


def train_model():
    parser = argparse.ArgumentParser()

    parser.add_argument('dataset', help='Path to directory containing train.txt, val.txt, and mean.npy')
    parser.add_argument('images',  help='Root directory of input images')
    parser.add_argument('labels',  help='Root directory of label images')
    
    parser.add_argument('--batchsize', '-b', type=int, default=16,
                        help='Number of images in each mini-batch')
    parser.add_argument('--test-batchsize', '-B', type=int, default=4,
                        help='Number of images in each test mini-batch')
    parser.add_argument('--epoch', '-e', type=int, default=50,
                        help='Number of sweeps over the dataset to train')
    parser.add_argument('--frequency', '-f', type=int, default=1,
                        help='Frequency of taking a snapshot')
    parser.add_argument('--gpu', '-g', type=int, default=0,
                        help='GPU ID (negative value indicates CPU)') #NOTE: function removed
    parser.add_argument('--out', '-o', default='logs',
                        help='Directory to output the result under "models" directory')
    parser.add_argument('--resume', '-r', default='',
                        help='Resume the training from snapshot')
    parser.add_argument('--noplot', dest='plot', action='store_false',
                        help='Disable PlotReport extension')

    parser.add_argument('--tcrop', type=int, default=400,
                        help='Crop size for train-set images')
    parser.add_argument('--vcrop', type=int, default=480,
                        help='Crop size for validation-set images')

    args = parser.parse_args()

    assert (args.tcrop % 16 == 0) and (args.vcrop % 16 == 0), "tcrop and vcrop must be divisible by 16."


    from tboard_logger import TensorboardLogger

    print('GPU: {}'.format(args.gpu))
    print('# Minibatch-size: {}'.format(args.batchsize))
    print('# Crop-size: {}'.format(args.tcrop))
    print('# epoch: {}'.format(args.epoch))
    print('')
    
    this_dir = os.path.dirname(os.path.abspath(__file__))
    models_dir = os.path.normpath(os.path.join(this_dir, "../../models"))
    log_dir = os.path.join(models_dir, args.out)
    writer = SummaryWriter(log_dir=log_dir)
    
    # Set up a neural network to train
    # Classifier reports softmax cross entropy loss and accuracy at every
    # iteration, which will be used by the PrintReport extension below.
    model = UNet()
    if args.gpu >= 0:
        # Make a specified GPU current
        chainer.cuda.get_device_from_id(args.gpu).use()
        model.to_gpu()  # Copy the model to the GPU

    # Setup an optimizer
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(model)
    
    # Load mean image
    mean = np.load(os.path.join(args.dataset, "mean.npy"))
    
    # Load the MNIST dataset
    train = LabeledImageDataset(os.path.join(args.dataset, "train.txt"), args.images, args.labels, 
                                mean=mean, crop_size=args.tcrop, test=False, distort=False)
    
    test = LabeledImageDataset (os.path.join(args.dataset, "val.txt"), args.images, args.labels, 
                                mean=mean, crop_size=args.vcrop, test=True, distort=False)

    train_iter = chainer.iterators.SerialIterator(train, args.batchsize)
    test_iter = chainer.iterators.SerialIterator(test, args.test_batchsize, repeat=False, shuffle=False)

    # Set up a trainer
    updater = training.StandardUpdater(
        train_iter, optimizer, device=args.gpu)
    trainer = training.Trainer(updater, (args.epoch, 'epoch'), out=log_dir)

    # Evaluate the model with the test dataset for each epoch
    trainer.extend(extensions.Evaluator(test_iter, model, device=args.gpu))

    # Dump a computational graph from 'loss' variable at the first iteration
    # The "main" refers to the target link of the "main" optimizer.
    trainer.extend(extensions.dump_graph('main/loss'))

    # Take a snapshot for each specified epoch
    frequency = args.epoch if args.frequency == -1 else max(1, args.frequency)
    trainer.extend(extensions.snapshot(), trigger=(frequency, 'epoch'))
    
    # Save trained model for each specific epoch
    trainer.extend(extensions.snapshot_object(
        model, 'model_iter_{.updater.iteration}'), trigger=(frequency, 'epoch'))

    # Write a log of evaluation statistics for each epoch
    trainer.extend(extensions.LogReport())

    # Save two plot images to the result dir
    if args.plot and extensions.PlotReport.available():
        trainer.extend(
            extensions.PlotReport(['main/loss', 'validation/main/loss'],
                                  'epoch', file_name='loss.png'))
        trainer.extend(
            extensions.PlotReport(
                ['main/accuracy', 'validation/main/accuracy'],
                'epoch', file_name='accuracy.png'))

    # Print selected entries of the log to stdout
    # Here "main" refers to the target link of the "main" optimizer again, and
    # "validation" refers to the default name of the Evaluator extension.
    # Entries other than 'epoch' are reported by the Classifier link, called by
    # either the updater or the evaluator.
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

    # Print a progress bar to stdout
    trainer.extend(extensions.ProgressBar())
    
    # Write training log to TensorBoard log file
    trainer.extend(TensorboardLogger(writer,
        ['main/loss', 'validation/main/loss',
         'main/accuracy', 'validation/main/accuracy']))
    
    if args.resume:
        # Resume from a snapshot
        chainer.serializers.load_npz(args.resume, trainer)

    # Run the training
    trainer.run()


if __name__ == '__main__':
    train_model()

Overwriting localisation_model.py


In [ ]:
!curl https://colab.chainer.org/install | CHAINER_VERSION="==5.1.0" CUPY_VERSION="==5.1.0" sh -
!pip install tensorboardx

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   8586      0 --:--:-- --:--:-- --:--:--  8586
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
+ pip install -q cupy-cuda100 ==5.1.0 chainer ==5.1.0
     |████████████████████████████████| 298.6 MB 13 kB/s 
     |████████████████████████████████| 513 kB 55.4 MB/s 
+ set +ex
Installation succeeded!
     |████████████████████████████████| 125 kB 14.4 MB/s 


In [ ]:
#Set base path for python to retrieve required accessories
%cd "/content/drive/MyDrive/xView2_baseline-master/spacenet/src/models"

/content/drive/MyDrive/xView2_baseline-master/spacenet/src/models


In [ ]:
#Check returns path/to/spacenet/src/models
!pwd

/content/drive/MyDrive/xView2_baseline-master/spacenet/src/models


In [ ]:
!python3 localisation_model.py "/content/drive/MyDrive/spacenet_gt/dataSet" "/content/drive/MyDrive/spacenet_gt/images" "/content/drive/MyDrive/spacenet_gt/labels" -e 50 -o "/content/drive/MyDrive/Model_data/localisation_model"

#Damage Classification Model

In [ ]:
#Reset path to base
%cd "/content"

/content


In [ ]:
%%writefile process_data.py
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict
from sklearn.model_selection import train_test_split
logging.basicConfig(level=logging.INFO)

# Configurations
NUM_WORKERS = 4
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 120
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_STEP = 150

damage_intensity_encoding = defaultdict(lambda: 0)
damage_intensity_encoding['destroyed'] = 3
damage_intensity_encoding['major-damage'] = 2
damage_intensity_encoding['minor-damage'] = 1
damage_intensity_encoding['no-damage'] = 0


def process_img(img_array, polygon_pts, scale_pct):
    """Process Raw Data into

            Args:
                img_array (numpy array): numpy representation of image.
                polygon_pts (array): corners of the building polygon.

            Returns:
                numpy array: .

    """

    height, width, _ = img_array.shape

    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
    xmin = max(int(xmin - (xdiff * scale_pct)), 0)
    xmax = min(int(xmax + (xdiff * scale_pct)), width)
    ymin = max(int(ymin - (ydiff * scale_pct)), 0)
    ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[ymin:ymax, xmin:xmax, :]


def process_data(input_path, output_path, output_csv_path, val_split_pct):
    """Process Raw Data into

        Args:
            dir_path (path): Path to the xBD dataset.
            data_type (string): String to indicate whether to process
                                train, test, or holdout data.

        Returns:
            x_data: A list of numpy arrays representing the images for training
            y_data: A list of labels for damage represented in matrix form

    """
    x_data = []
    y_data = []

    disasters = [folder for folder in os.listdir(input_path) if not folder.startswith('.')]
    disaster_paths = ([input_path + "/" +  d + "/images" for d in disasters])
    image_paths = []
    image_paths.extend([(disaster_path + "/" + pic) for pic in os.listdir(disaster_path)] for disaster_path in disaster_paths)
    img_paths = np.concatenate(image_paths)

    for img_path in tqdm(img_paths):

        img_obj = Image.open(img_path)
        img_array = np.array(img_obj)

        #Get corresponding label for the current image
        label_path = img_path.replace('png', 'json').replace('images', 'labels')
        label_file = open(label_path)
        label_data = json.load(label_file)

        for feat in label_data['features']['xy']:

            # only images post-disaster will have damage type
            try:
                damage_type = feat['properties']['subtype']
            except: # pre-disaster damage is default no-damage
                damage_type = "no-damage"
                continue

            poly_uuid = feat['properties']['uid'] + ".png"

            y_data.append(damage_intensity_encoding[damage_type])

            polygon_geom = shapely.wkt.loads(feat['wkt'])
            polygon_pts = np.array(list(polygon_geom.exterior.coords))
            poly_img = process_img(img_array, polygon_pts, 0.8)
            cv2.imwrite(output_path + "/" + poly_uuid, poly_img)
            x_data.append(poly_uuid)
    
    output_train_csv_path = os.path.join(output_csv_path, "train.csv")

    if(val_split_pct > 0):
       x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=val_split_pct)
       data_array_train = {'uuid': x_train, 'labels': y_train}
       data_array_test = {'uuid': x_test, 'labels': y_test}
       output_test_csv_path = os.path.join(output_csv_path, "test.csv")
       df_train = pd.DataFrame(data_array_train)
       df_test = pd.DataFrame(data_array_test)
       df_train.to_csv(output_train_csv_path)
       df_test.to_csv(output_test_csv_path)
    else: 
       data_array = {'uuid': x_data, 'labels': y_data}
       df = pd.DataFrame(data = data_array)
       df.to_csv(output_train_csv_path)
    

def main():

    parser = argparse.ArgumentParser(description='Run Building Damage Classification Training & Evaluation')
    parser.add_argument('--input_dir',
                        required=True,
                        metavar="/path/to/xBD_input",
                        help="Full path to the parent dataset directory")
    parser.add_argument('--output_dir',
                        required=True,
                        metavar='/path/to/xBD_output',
                        help="Path to new directory to save images")
    parser.add_argument('--output_dir_csv',
                        required=True,
                        metavar='/path/to/xBD_output_csv',
                        help="Path to new directory to save csv")
    parser.add_argument('--val_split_pct', 
                        required=False,
                        default=0.0,
                        metavar='Percentage to split validation',
                        help="Percentage to split ")
    args = parser.parse_args()

    logging.info("Started Processing for Data")
    process_data(args.input_dir, args.output_dir, args.output_dir_csv, float(args.val_split_pct))
    logging.info("Finished Processing Data")


if __name__ == '__main__':
    main()
 

 

Writing process_data.py


In [ ]:
!python3 process_data.py --input_dir "/content/drive/MyDrive/Model_data/xBD" --output_dir "/content/drive/MyDrive/Model_data/test-train2" --output_dir_csv "/content/drive/MyDrive/Model_data/output_csv" --val_split_pct .5

INFO:root:Started Processing for Data
100% 240/240 [03:37<00:00,  1.10it/s]
INFO:root:Finished Processing Data


In [ ]:
#From the test.csv file, file names are copied into txt to be moved to test folder using code below. Repeated for training files.
from pathlib import Path
import shutil
import os

file_source ='/content/drive/MyDrive/Model_data/test-train2'
dst ='/content/drive/MyDrive/Model_data/test-train2/test'

with open('/content/drive/MyDrive/Model_data/testset.txt') as my_file:
    for filename in my_file:
        src = os.path.join(file_source, filename.strip() )
        os.rename(src, os.path.join(dst, filename.strip()))

In [ ]:
from pathlib import Path
import shutil
import os

file_source ='/content/drive/MyDrive/Model_data/test-train2'
dst ='/content/drive/MyDrive/Model_data/test-train2/train'

with open('/content/drive/MyDrive/Model_data/trainset.txt') as my_file:
    for filename in my_file:
        src = os.path.join(file_source, filename.strip() )
        os.rename(src, os.path.join(dst, filename.strip()))

In [ ]:
%%writefile damage_classification.py


from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
print(tf.__version__)
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import backend as K



###
# Loss function for ordinal loss from https://github.com/JHart96/keras_ordinal_categorical_crossentropy
###
def ordinal_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * keras.losses.categorical_crossentropy(y_true, y_pred )


###
# Generate a simple CNN
###
def generate_xBD_baseline_model():
  weights = 'imagenet'
  inputs = Input(shape=(128, 128, 3))

  base_model = ResNet50(include_top=False, weights=weights, input_shape=(128, 128, 3))

  for layer in base_model.layers:
    layer.trainable = False

  x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Flatten()(x)

  base_resnet = base_model(inputs)
  base_resnet = Flatten()(base_resnet)

  concated_layers = Concatenate()([x, base_resnet])

  concated_layers = Dense(2024, activation='relu')(concated_layers)
  concated_layers = Dense(524, activation='relu')(concated_layers)
  concated_layers = Dense(124, activation='relu')(concated_layers)
  output = Dense(3, activation='relu')(concated_layers)

  model = Model(inputs=inputs, outputs=output)

  return model



logging.basicConfig(level=logging.INFO)

# Configurations
NUM_WORKERS = 1 
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 30
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_STEP = 400
LOG_DIR = '/path/to/logs' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

damage_intensity_encoding = dict()
damage_intensity_encoding[3] = '3'
damage_intensity_encoding[2] = '2' 
damage_intensity_encoding[1] = '1' 
damage_intensity_encoding[0] = '0' 


# Function to compute unweighted f1 scores, just for reference
###


def f1_unweighted(y_true, y_pred):
    def recall_unweighted(y_true, y_pred):
        """Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives_un = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives_un = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall_unweighted = true_positives_un / (possible_positives_un + K.epsilon())
        return recall_unweighted

    def precision_unweighted(y_true, y_pred):
        """Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives_un = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives_un = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision_unweighted = true_positives_un / (predicted_positives_un + K.epsilon())
        return precision_unweighted

    precision_unweighted = precision_unweighted(y_true, y_pred)
    recall_unweighted = recall_unweighted(y_true, y_pred)
    return 2*((precision_unweighted*recall_unweighted)/(precision_unweighted+recall_unweighted+K.epsilon()))


###
# Creates data generator for validation set
###
def validation_generator(test_csv, test_dir):
    df = pd.read_csv(test_csv)
    df = df.replace({"labels" : damage_intensity_encoding })

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1/255.)


    return gen.flow_from_dataframe(dataframe=df,
                                   directory=test_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


###
# Applies random transformations to training data
###
def augment_data(df, in_dir):

    df = df.replace({"labels" : damage_intensity_encoding })
    gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             rescale=1/255.)
    return gen.flow_from_dataframe(dataframe=df,
                                   directory=in_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


# Run training and evaluation based on existing or new model
def train_model(train_data, train_csv, test_data, test_csv, model_in, model_out):

    model = generate_xBD_baseline_model()

    # Add model weights if provided by user
    if model_in is not None:
        model.load_weights(model_in)

    df = pd.read_csv(train_csv)
    class_weights = compute_class_weight(
        class_weight = "balanced",
        classes = np.unique(df['labels'].to_list()),
        y = df['labels'].to_list());
    d_class_weights = dict(enumerate(class_weights))
    print('Class weights'+str(d_class_weights))

    samples = df['uuid'].count()
    steps = np.ceil(samples/BATCH_SIZE)

    # Augments the training data
    train_gen_flow = augment_data(df, train_data)

    #Set up tensorboard logging
    tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir=LOG_DIR,
                                                        batch_size=BATCH_SIZE)

    
    #Filepath to save model weights
    filepath = model_out + "-saved-model-{epoch:02d}-{accuracy:.2f}.hdf5"
    checkpoints = keras.callbacks.ModelCheckpoint(filepath,
                                                    monitor=['loss', 'accuracy'],
                                                    verbose=1,
                                                    save_best_only=False,
                                                    mode='max')

    #Adds adam optimizer
    adam = tf.keras.optimizers.Adam(lr=LEARNING_RATE,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    decay=0.0,
                                    amsgrad=False)
    
        #Defines the metrics to be used
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve        
    ]
    
    #Defines the metrics to be used
    model.compile(loss=ordinal_loss, optimizer=adam, metrics=['accuracy', f1_unweighted, METRICS])
    model.summary

    # Augments the training and validaton data
    train_ds = augment_data(df, train_data)
    validation_dataset = validation_generator(test_csv, test_data)

    #Training begins
    history = model.fit_generator(generator=train_ds,
                        validation_data=validation_dataset,
                        epochs=NUM_EPOCHS,
                        workers=NUM_WORKERS,
                        use_multiprocessing=True,
                        class_weight=d_class_weights,
                        callbacks=[tensorboard_callbacks, checkpoints],
                        verbose=1)


    #Evalulate f1 weighted scores on validation set
    validation_gen = validation_generator(test_csv, test_data)
    predictions = model.predict(validation_gen)

    val_trues = validation_gen.classes
    val_pred = np.argmax(predictions, axis=-1)

    f1_weighted = f1_score(val_trues, val_pred, average='weighted')
    print('validation f1 score: ' +str(f1_weighted))


    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(NUM_EPOCHS)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()


def main():

    parser = argparse.ArgumentParser(description='Run Building Damage Classification Training & Evaluation')
    parser.add_argument('--train_data',
                        required=True,
                        metavar="/path/to/xBD_train",
                        help="Full path to the train data directory")
    parser.add_argument('--train_csv',
                        required=True,
                        metavar="/path/to/xBD_split",
                        help="Full path to the train csv")
    parser.add_argument('--test_data',
                        required=True,
                        metavar="/path/to/xBD_test",
                        help="Full path to the test data directory")
    parser.add_argument('--test_csv',
                        required=True,
                        metavar="/path/to/xBD_split",
                        help="Full path to the test csv")
    parser.add_argument('--model_in',
                        default=None,
                        metavar='/path/to/input_model',
                        help="Path to save model")
    parser.add_argument('--model_out',
                        required=True,
                        metavar='/path/to/save_model',
                        help="Path to save model")

    args = parser.parse_args()

    train_model(args.train_data, args.train_csv, args.test_data, args.test_csv, args.model_in, args.model_out)


if __name__ == '__main__':
    main()


Writing damage_classification.py


In [ ]:
!python3 damage_classification.py --train_data "/content/drive/MyDrive/Model_data/test-train" --train_csv "/content/drive/MyDrive/Model_data/output_csv/train_skewed.csv" --test_data "/content/drive/MyDrive/Model_data/test-train" --test_csv "/content/drive/MyDrive/Model_data/output_csv/test.csv" --model_out "/content/drive/MyDrive/Model_data/saved_models/pre_test/pre-recall"

#Inference

Combing spacenet (localization) model and classification model

In [ ]:
!curl https://colab.chainer.org/install | CHAINER_VERSION="==5.1.0" CUPY_VERSION="==5.1.0" sh -
!pip install imantics
!pip install simplification

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   7348      0 --:--:-- --:--:-- --:--:--  7348
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
+ pip install -q cupy-cuda100 ==5.1.0 chainer ==5.1.0
ERROR: Operation cancelled by user
^C
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16033 sha256=e9faf58f1db02d6cfd8991e9efd27dcdfbc630704df9f56ad1733a47b2fbb689
  Stored in directory: /root/.cache/pip/wheels/da/7c/3e/296fe3ed4eb3bd713e91dee0d0549f12f316d4

In [ ]:
#Creating directories
%%shell
set -euo pipefail

# this function is called when Ctrl-C is sent
function trap_ctrlc ()
{
    # perform cleanup here
    echo "Ctrl-C or Error caught...performing clean up check /tmp/inference.log"

    if [ -d /tmp/inference ]; then
           rm -rf /tmp/inference
    fi

    exit 99
}

# initialise trap to call trap_ctrlc function
# when signal 2 (SIGINT) is received
trap "trap_ctrlc" 2 9 13 3

help_message () {
        printf "${0}: Runs the polygonization in inference mode\n\t-x: path to xview-2 repository\n\t-i: /full/path/to/input/pre-disaster/image.png\n\t-p: /full/path/to/input/post-disaster/image.png\n\t-o: /path/to/output.png\n\t-l: path/to/localization_weights\n\t-c: path/to/classification_weights\n\t-e /path/to/virtual/env/activate\n\t-y continue with local environment and without interactive prompt\n\n"
}

input="/content/drive/MyDrive/mexico_pre/mexico-earthquake_00000002_pre_disaster.png"
input_post="/content/drive/MyDrive/mexico_post/mexico-earthquake_00000002_post_disaster.png"
inference_base="/tmp/inference"
LOGFILE="/tmp/inference_log"
XBDIR="/content/drive/MyDrive/xView2_baseline-master"
virtual_env="/content/drive/MyDrive/xView2_baseline-master/bin/activate"
localization_weights="/content/drive/MyDrive/localization.h5"
classification_weights="/content/drive/MyDrive/Model_data/saved_models/Model_data-saved-model-86-0.70.hdf5"
continue_answer="y"


# Create the output directory if it doesn't exist 
mkdir -p "$inference_base"

if ! [ -f "$LOGFILE" ]; then
    touch "$LOGFILE"
fi

printf "==========\n" >> "$LOGFILE"
echo `date +%Y%m%dT%H%M%S` >> "$LOGFILE"
printf "\n" >> "$LOGFILE"

input_image=${input##*/}

label_temp="$inference_base"/"${input_image%.*}"/labels
mkdir -p "$label_temp"

printf "\n"

printf "\n"

# Run in inference mode
# Because of the models _have_ to be in the correct directory, they use relative paths to find the source (e.g. "../src") 
# sourcing the virtual environment packages if they exist
# this is *necessary* or all packages must be installed globally
if [ -f  "$virtual_env" ]; then
    source "$virtual_env"
else
    if [ "$continue_answer" = "n" ]; then 
        printf "Error: cannot source virtual environment  \n\tDo you have all the dependencies installed and want to continue? [Y/N]: "
        read continue_answer 
        if [ "$continue_answer" == "N" ]; then 
               exit 2
        fi 
    fi
fi

cd "$XBDIR"/model

printf "Grabbing post image file for classification\n"
disaster_post_file="$input_post"

mkdir -p "$inference_base"/output_polygons

printf "Running classification\n" 



Grabbing post image file for classification
Running classification


In [ ]:
%%writefile process_data_inference.py

from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict


def process_img(img_array, polygon_pts, scale_pct):
    """Process Raw Data into
            Args:
                img_array (numpy array): numpy representation of image.
                polygon_pts (array): corners of the building polygon.
            Returns:
                numpy array: extracted polygon image from img_array.
    """

    height, width, _ = img_array.shape

    #Find the four corners of the polygon
    xcoords = polygon_pts[:, 0]
    ycoords = polygon_pts[:, 1]
    xmin, xmax = np.min(xcoords), np.max(xcoords)
    ymin, ymax = np.min(ycoords), np.max(ycoords)

    xdiff = xmax - xmin
    ydiff = ymax - ymin

    #Extend image by scale percentage
    xmin = max(int(xmin - (xdiff * scale_pct)), 0)
    xmax = min(int(xmax + (xdiff * scale_pct)), width)
    ymin = max(int(ymin - (ydiff * scale_pct)), 0)
    ymax = min(int(ymax + (ydiff * scale_pct)), height)

    return img_array[ymin:ymax, xmin:xmax, :]


def process_img_poly(img_path, label_path,  output_dir, output_csv):
    x_data = [] 
    img_obj = Image.open(img_path)

    #Applies histogram equalization to image
    img_array = np.array(img_obj)
    #clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    #img_array = clahe.apply(img_array_pre)

    #Get corresponding label for the current image
    label_file = open(label_path)
    label_data = json.load(label_file)

    #Find all polygons in a given image
    for feat in label_data['features']['xy']:

        poly_uuid = feat['properties']['uid'] + ".png"

        # Extract the polygon from the points given
        polygon_geom = shapely.wkt.loads(feat['wkt'])
        polygon_pts = np.array(list(polygon_geom.exterior.coords))
        poly_img = process_img(img_array, polygon_pts, 0.8)

        # Write out the polygon in its own image
        cv2.imwrite(output_dir + "/" + poly_uuid, poly_img)
        x_data.append(poly_uuid)

    data_array = {'uuid': x_data}
    df = pd.DataFrame(data = data_array)
    df.to_csv(output_csv)

def main():

    parser = argparse.ArgumentParser(description='Run Building Damage Classification Training & Evaluation')
    parser.add_argument('--input_img',
                        required=True,
                        metavar="/path/to/xBD_input",
                        help="Full path to the parent dataset directory")
    parser.add_argument('--label_path',
                        required=True,
                        metavar="/path/to/xBD_input",
                        help="Full path to the parent dataset directory")
    parser.add_argument('--output_dir',
                        required=True,
                        metavar='/path/to/xBD_output',
                        help="Path to new directory to save images")
    parser.add_argument('--output_csv',
                        required=True, 
                        metavar='/path/to/xBD_output',
                        help="Path to save the csv file")

    args = parser.parse_args()

    process_img_poly(args.input_img, args.label_path, args.output_dir, args.output_csv)


if __name__ == '__main__':
    main()

Writing process_data_inference.py


In [ ]:
#Using weights from pre-trained localisation model
#If using locally trained localisation model, run Inference.py provide in bin to generate json file

!python3 process_data_inference.py --input_img "/content/drive/MyDrive/Model_data/xBD/mexico-earthquake/images/mexico-earthquake_00000002_post_disaster.png" --label_path "/content/drive/MyDrive/Model_data/xBD/mexico-earthquake/labels/mexico-earthquake_00000002_pre_disaster.json" --output_dir "/tmp/inference/output_polygons" --output_csv "/tmp/inference/output.csv"

In [ ]:
%%writefile damage_inference.py
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
from sys import exit
import cv2
import datetime

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import backend as K

###
# Loss function for ordinal loss from https://github.com/JHart96/keras_ordinal_categorical_crossentropy
###
def ordinal_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * keras.losses.categorical_crossentropy(y_true, y_pred )


###
# Generate a simple CNN
###
def generate_xBD_baseline_model():
  weights = 'imagenet'
  inputs = Input(shape=(128, 128, 3))

  base_model = ResNet50(include_top=False, weights=weights, input_shape=(128, 128, 3))

  for layer in base_model.layers:
    layer.trainable = False

  x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Flatten()(x)

  base_resnet = base_model(inputs)
  base_resnet = Flatten()(base_resnet)

  concated_layers = Concatenate()([x, base_resnet])

  concated_layers = Dense(2024, activation='relu')(concated_layers)
  concated_layers = Dense(524, activation='relu')(concated_layers)
  concated_layers = Dense(124, activation='relu')(concated_layers)
  output = Dense(3, activation='relu')(concated_layers)

  model = Model(inputs=inputs, outputs=output)

  return model


# Configurations
NUM_WORKERS = 4
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 120
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_DIR = '/tmp/inference/classification_log_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


damage_intensity_encoding = dict() 
damage_intensity_encoding[3] = 'destroyed' 
damage_intensity_encoding[2] = 'major-damage'
damage_intensity_encoding[1] = 'minor-damage'
damage_intensity_encoding[0] = 'no-damage'


###
# Creates data generator for validation set
###
def create_generator(test_df, test_dir, output_json_path):

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1.4)

    try:
        gen_flow = gen.flow_from_dataframe(dataframe=test_df,
                                   directory=test_dir,
                                   x_col='uuid',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode=None,
                                   target_size=(128, 128))
    except:
        # No polys detected so write out a blank json
        blank = {}
        with open(output_json_path , 'w') as outfile:
            json.dump(blank, outfile)
        exit(0)


    return gen_flow

# Runs inference on given test data and pretrained model
def run_inference(test_data, test_csv, model_weights, output_json_path):

   model = generate_xBD_baseline_model()
   model.load_weights(model_weights)

   adam = tf.keras.optimizers.Adam(lr=LEARNING_RATE,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    epsilon=None,
                                    decay=0.0,
                                    amsgrad=False)


   model.compile(loss=ordinal_loss, optimizer=adam, metrics=['accuracy'])

   df = pd.read_csv(test_csv)

   test_gen = create_generator(df, test_data, output_json_path)
   test_gen.reset()
   samples = df["uuid"].count()

   steps = np.ceil(samples/BATCH_SIZE)

   tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)

   predictions = model.predict_generator(generator=test_gen,
                    callbacks=[tensorboard_callbacks],
                    verbose=1)

   predicted_indices = np.argmax(predictions, axis=1)
   predictions_json = dict()
   for i in range(samples):
       filename_raw = test_gen.filenames[i]
       filename = filename_raw.split(".")[0]
       predictions_json[filename] = damage_intensity_encoding[predicted_indices[i]]

   with open(output_json_path , 'w') as outfile:
       json.dump(predictions_json, outfile)


def main():

    parser = argparse.ArgumentParser(description='Run Building Damage Classification Training & Evaluation')
    parser.add_argument('--test_data',
                        required=True,
                        metavar="/path/to/xBD_test_dir",
                        help="Full path to the parent dataset directory")
    parser.add_argument('--test_csv',
                        required=True,
                        metavar="/path/to/xBD_test_csv",
                        help="Full path to the parent dataset directory")
    parser.add_argument('--model_weights',
                        default=None,
                        metavar='/path/to/input_model_weights',
                        help="Path to input weights")
    parser.add_argument('--output_json',
                        required=True,
                        metavar="/path/to/output_json")

    args = parser.parse_args()

    run_inference(args.test_data, args.test_csv, args.model_weights, args.output_json)


if __name__ == '__main__':
    main()

Writing damage_inference.py


In [ ]:
#Generating JSON from damager classification model
!python3 damage_inference.py --test_data "/tmp/inference/output_polygons" --test_csv "/tmp/inference/output.csv" --model_weights "/content/drive/MyDrive/Model_data/saved_models/Model_data-saved-model-86-0.70.hdf5" --output_json "/tmp/inference/classification_inference.json"

2022-05-02 16:23:35.355590: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Traceback (most recent call last):
  File "damage_inference.py", line 188, in <module>
    main()
  File "damage_inference.py", line 184, in main
    run_inference(args.test_data, args.test_csv, args.model_weights, args.output_json)
  File "damage_inference.py", line 126, in run_inference
    model.load_weights(model_weights)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.7/dist-packages/h5py/_hl/files.py", line 427, in __init__
    swmr=swmr)
  File "/usr/local/lib/python3.7/dist-packages/h5py/_hl/files.py", line 190, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phi

In [ ]:
%%writefile combine_jsons.py

import json 

def combine_output(pred_polygons, pred_classification, output_file):
    """
    :param pred_polygons: the file path to the localization inference output json  
    :param pre_classification: the file path to the classification inference output json
    :param output_file: the file path to store the combined json file
    """

    # Skeleton of the json with null values 
    output_json = {
        "features": {
            "lng_lat": [],
            "xy": []
        }, 
        "metadata": {
            "sensor": "",
            "provider_asset_type": "",
            "gsd": 0,
            "capture_date": "", 
            "off_nadir_angle": 0, 
            "pan_resolution": 0, 
            "sun_azimuth": 0, 
            "sun_elevation": 0, 
            "target_azimuth": 0, 
            "disaster": "", 
            "disaster_type": "", 
            "catalog_id": "", 
            "original_width": 0, 
            "original_height": 0, 
            "width": 0, 
            "height": 0, 
            "id": "", 
            "img_name": ""
        }
    }

    # Open the classification json 
    with open(pred_classification) as labels:
        label_json = json.load(labels)
        
        # Open the localization json 
        with open(pred_polygons) as polys:
            poly_json = json.load(polys)

            # Match UUIDs from the two jsons and combine in output_json skeleton 
            for p in poly_json['features']['xy']:
                p['properties']['subtype'] = label_json[p['properties']['uid']]
                output_json['features']['xy'].append(p)
    
    # Finally save out the combined json file 
    with open(output_file, 'w') as out: 
        json.dump(output_json, out)

if __name__ == '__main__': 
    import argparse

    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description=
        """combine_jsons.py: combines the outputs of localization and classification inference into a single output json"""
    )
    parser.add_argument('--polys',
                        required=True,
                        metavar='/path/to/input/polygons.json',
                        help="Full path to the json from polygonize.py")
    parser.add_argument('--classes',
                        required=True,
                        metavar='/path/to/classifications.json',
                        help="Full path to the json from tensor_inf.py"
    )
    parser.add_argument('--output',
                        required=True,
                        metavar='/path/to/pred.json',
                        help="Full path to save the final single output file to"
    )

    args = parser.parse_args()

    # Combining the json based off the uuid assigned at the polygonize stage
    combine_output(args.polys, args.classes, args.output)

Writing combine_jsons.py


In [ ]:
#If running locaization locally, replace --polys with json produced by Inference.py
!python3 combine_jsons.py --polys "/content/drive/MyDrive/Model_data/xBD/mexico-earthquake/labels/mexico-earthquake_00000002_pre_disaster.json" --classes "/tmp/inference/classification_inference.json" --output "/tmp/inference/inference.json"

Traceback (most recent call last):
  File "combine_jsons.py", line 82, in <module>
    combine_output(args.polys, args.classes, args.output)
  File "combine_jsons.py", line 40, in combine_output
    with open(pred_classification) as labels:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/inference/classification_inference.json'


In [ ]:
%%writefile inference_image_output.py

import json
from shapely import wkt
from shapely.geometry import Polygon
import numpy as np 
from cv2 import fillPoly, imwrite

def open_json(json_file_path):
    """
    :param json_file_path: path to open inference json file
    :returns: the json data dictionary of localized polygon and their classifications 
    """

    with open(json_file_path) as jf:
        json_data = json.load(jf)
        inference_data = json_data['features']['xy']
        return inference_data

def create_image(inference_data):
    """
    :params inference_data: json data dictionary of localized polygon and their classifications
    :returns: an numpy array of 8-bit grey scale image with polygons filled in according to the key provided
    """

    damage_key = {'un-classified': 1, 'no-damage': 1, 'minor-damage': 2, 'major-damage': 3, 'destroyed': 4}

    mask_img = np.zeros((1024,1024,1), np.uint8)

    for poly in inference_data:
        damage = poly['properties']['subtype']
        coords = wkt.loads(poly['wkt'])
        poly_np = np.array(coords.exterior.coords, np.int32)
        
        fillPoly(mask_img, [poly_np], damage_key[damage])
    
    return mask_img

def save_image(polygons, output_path):
    """
    :param polygons: np array with filled in polygons from create_image()
    :param output_path: path to save the final output inference image
    """

    # Output the filled in polygons to an image file
    imwrite(output_path, polygons)
  
def create_inference_image(json_input_path, image_output_path):
    """
    :param json_input_path: Path to output inference json file
    :param image_outut_pat: Path to save the final inference image
    """

    # Getting the inference data from the localization and classification 
    inference_data = open_json(json_input_path)

    # Filling in the polygons and readying the image format 
    polygon_array = create_image(inference_data)

    # Saving the image to the desired location
    save_image(polygon_array, image_output_path)

if __name__ == '__main__': 
    import argparse

    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description=
        """inference_image_output.py: Takes the inference localization and classification final outputs in json from and outputs an image ready to be scored based off the challenge parameters""")
    parser.add_argument('--input',
                        required=True,
                        metavar='/path/to/final/inference.json',
                        help="Full path to the final inference json")
    parser.add_argument('--output',
                        required=True,
                        metavar='/path/to/inference.png',
                        help="Full path to save the image to")

    args = parser.parse_args()

    # Creating the scoring image
    create_inference_image(args.input, args.output)

Writing inference_image_output.py


In [ ]:
!python3 inference_image_output.py --input "/tmp/inference/inference.json" --output "/content/drive/MyDrive/Model_data/inference-output/baseline_output002.png"

Traceback (most recent call last):
  File "inference_image_output.py", line 81, in <module>
    create_inference_image(args.input, args.output)
  File "inference_image_output.py", line 54, in create_inference_image
    inference_data = open_json(json_input_path)
  File "inference_image_output.py", line 14, in open_json
    with open(json_file_path) as jf:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/inference/inference.json'


In [ ]:
%%writefile display_image.py

import numpy
from PIL import Image
import cv2


def get_image(input_image, image_output):
  image = cv2.imread(input_image)
  gray = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

  cv2.imshow('Original image',image)
  cv2.imshow('Gray image', gray)



if __name__ == '__main__': 
    import argparse

    parser = argparse.ArgumentParser(description='Convert Image to RGBA')
    parser.add_argument('--input_image',
                        required=True,
                        metavar="/path/to/inference-output/image.png",
                        help="Full path to the inference output image to be converted")
    parser.add_argument('--image_output',
                        required=True,
                        metavar="/path/to/output_directory",
                        help="Full path to the ilocation of the output image")

    args = parser.parse_args()

    get_image(args.input_image, args.image_output)




Writing display_image.py


In [ ]:
!python3 display_image.py --input_image "/content/drive/MyDrive/Model_data/inference-output/baseline_output002.png" --image_output "/content/drive/MyDrive/Model_data/inference-output"

Traceback (most recent call last):
  File "display_image.py", line 31, in <module>
    get_image(args.input_image, args.image_output)
  File "display_image.py", line 9, in get_image
    gray = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
cv2.error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function 'cv::impl::{anonymous}::CvtHelper<VScn, VDcn, VDepth, sizePolicy>::CvtHelper(cv::InputArray, cv::OutputArray, int) [with VScn = cv::impl::{anonymous}::Set<1>; VDcn = cv::impl::{anonymous}::Set<3, 4>; VDepth = cv::impl::{anonymous}::Set<0, 2, 5>; cv::impl::{anonymous}::SizePolicy sizePolicy = (cv::impl::<unnamed>::SizePolicy)2u; cv::InputArray = const cv::_InputArray&; cv::OutputArray = const cv::_OutputArray&]'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 3



In [ ]:
#Removing temporary directories
%%shell
printf "Cleaning up\n"
rm -rf "$inference_base"

printf "==========\n" >> "$LOGFILE"
printf "Done!\n"

Cleaning up
/bin/bash: line 3: : No such file or directory
Done!


# BIN

In [ ]:
%%writefile inference.py

import resource
import sys
sys.path.append('/content/drive/MyDrive/xView2_baseline-master/spacenet/src/models')

import segmentation_cpu
from segmentation_cpu import SegmentationModel as Model
from os import path
from PIL import Image
import numpy as np
from uuid import uuid4
import json
from imantics import Polygons, Mask
from simplification.cutil import simplify_coords_vwp

def create_wkt(polygon):
    """
    :param polygon: a single polygon in the format [(x1,y1), (x2,y2), ...]
    :returns: a wkt formatted string ready to be put into the json 
    """
    wkt = 'POLYGON (('

    for coords in polygon:
        wkt += "{} {},".format(coords[0], coords[1])

    wkt = wkt[:-1] + '))'

    return wkt


def create_json(adjusted_polygons):
    """
    :param polygons: list of polygons in the format [(x1,y1), (x2,y2), ...]
    :returns: json with found and adjusted polygon pixel x,y values in WKT format
    """
    # Create a blank json that matched the labeler provided jsons with null or default values
    output_json = {
        "features": {
            "lng_lat": [],
            "xy": []
        }, 
        "metadata": {
            "sensor": "",
            "provider_asset_type": "",
            "gsd": 0,
            "capture_date": "", 
            "off_nadir_angle": 0, 
            "pan_resolution": 0, 
            "sun_azimuth": 0, 
            "sun_elevation": 0, 
            "target_azimuth": 0, 
            "disaster": "", 
            "disaster_type": "", 
            "catalog_id": "", 
            "original_width": 0, 
            "original_height": 0, 
            "width": 0, 
            "height": 0, 
            "id": "", 
            "img_name": ""
        }
    }

    # Using a lambda function to place the WKT string in the list of polygons 
    polygon_template = lambda poly, uuid: {
        'properties': {
            'feature_type': 'building',
            'uid': uuid
        },
        'wkt': poly
    }

    # For each adjusted polygon add the wkt for the polygon points
    for polygon in adjusted_polygons:
        wkt = create_wkt(polygon)
        uuid = gen_uuid()
        poly = polygon_template(wkt, uuid)
        output_json['features']['xy'].append(poly)

    return output_json

def gen_uuid():
    return str(uuid4())

def inference(image, score, output_file):
    building_score = score[1]
    
    building_mask_pred = (np.argmax(score, axis=0) == 1)
    polygons = Mask(building_mask_pred).polygons()
    
    new_predictions = []
    
    for poly in polygons:
        if len(poly) >= 3:
            f = poly.reshape(-1, 2)
            simplified_vw = simplify_coords_vwp(f, .3)
            if len(simplified_vw) > 2:
                    mpoly = []
                    # Rebuilding the polygon in the way that PIL expects the values [(x1,y1),(x2,y2)]
                    for i in simplified_vw:
                        mpoly.append((i[0], i[1]))
                    # Adding the first point to the last to close the polygon
                    mpoly.append((simplified_vw[0][0], simplified_vw[0][1]))
                    new_predictions.append(mpoly)
            
    # Creating the json with the predicted and then adjusted polygons
    output_json = create_json(new_predictions)
    
    with open(output_file, 'w') as out_file:
        json.dump(output_json, out_file)

if __name__ == "__main__": 
    import argparse

    # Parse command line arguments
    parser = argparse.ArgumentParser(
        description=
        """inference.py: takes an image and creates inferred polygons json off the VW algorithm and the unet model predictions"""
    )
    parser.add_argument('--input',
                        required=True,
                        metavar='/path/to/input/image.png')
    parser.add_argument(
        '--weights',
        required=True,
        metavar='/full/path/to/mode_iter_XXXX',
        help="Must be the output to a unet model weights trained for xView2"
    )
    parser.add_argument(
        '--mean',
        required=True,
        metavar='/full/path/to/mean.npy',
        help="a numpy data structure file that is the mean of the training images (found by running ./src/features/compute_mean.py)"
    )
    parser.add_argument('--output',
                        required=True,
                        metavar="/path/to/output/file.json")
    args = parser.parse_args()

    # Load trained model
    # Modify the paths based on your trained model location if needed.
    mean = np.load(args.mean)
    model = Model(args.weights, mean)
 
    image = np.array(Image.open(args.input))
    score = model.apply_segmentation(image)
    inference(image, score, args.output)

Overwriting inference.py


In [ ]:
%%shell
set -euo pipefail

# this function is called when Ctrl-C is sent
function trap_ctrlc ()
{
    # perform cleanup here
    echo "Ctrl-C or Error caught...performing clean up check /tmp/inference.log"

    if [ -d /tmp/inference ]; then
           rm -rf /tmp/inference
    fi

    exit 99
}

# initialise trap to call trap_ctrlc function
# when signal 2 (SIGINT) is received
trap "trap_ctrlc" 2 9 13 3

help_message () {
        printf "${0}: Runs the polygonization in inference mode\n\t-x: path to xview-2 repository\n\t-i: /full/path/to/input/pre-disaster/image.png\n\t-p: /full/path/to/input/post-disaster/image.png\n\t-o: /path/to/output.png\n\t-l: path/to/localization_weights\n\t-c: path/to/classification_weights\n\t-e /path/to/virtual/env/activate\n\t-y continue with local environment and without interactive prompt\n\n"
}

input="/content/drive/MyDrive/mexico_pre/mexico-earthquake_00000002_pre_disaster.png"
input_post="/content/drive/MyDrive/mexico_post/mexico-earthquake_00000002_post_disaster.png"
inference_base="/tmp/inference"
LOGFILE="/tmp/inference_log"
XBDIR="/content/drive/MyDrive/xView2_baseline-master"
virtual_env="/content/drive/MyDrive/xView2_baseline-master/bin/activate"
localization_weights="/content/drive/MyDrive/localization.h5"
classification_weights="/content/drive/MyDrive/Model_data/saved_models/Model_data-saved-model-86-0.70.hdf5"
continue_answer="y"


# Create the output directory if it doesn't exist 
mkdir -p "$inference_base"

if ! [ -f "$LOGFILE" ]; then
    touch "$LOGFILE"
fi

printf "==========\n" >> "$LOGFILE"
echo `date +%Y%m%dT%H%M%S` >> "$LOGFILE"
printf "\n" >> "$LOGFILE"

input_image=${input##*/}

label_temp="$inference_base"/"${input_image%.*}"/labels
mkdir -p "$label_temp"

printf "\n"

printf "\n"

# Run in inference mode
# Because of the models _have_ to be in the correct directory, they use relative paths to find the source (e.g. "../src") 
# sourcing the virtual environment packages if they exist
# this is *necessary* or all packages must be installed globally
if [ -f  "$virtual_env" ]; then
    source "$virtual_env"
else
    if [ "$continue_answer" = "n" ]; then 
        printf "Error: cannot source virtual environment  \n\tDo you have all the dependencies installed and want to continue? [Y/N]: "
        read continue_answer 
        if [ "$continue_answer" == "N" ]; then 
               exit 2
        fi 
    fi
fi

cd "$XBDIR"/spacenet/inference/

# Quietly running the localization inference to output a json with the predicted polygons from the supplied input image
printf "Running localization\n"
python3 ./inference.py --input "$input" --weights "$localization_weights" --mean "$XBDIR"/weights/mean.npy --output "$label_temp"/"${input_image%.*}".json >> "$LOGFILE" 2>&1

printf "\n" >> "$LOGFILE"

# Classification inferences start below
cd "$XBDIR"/model

# Replace the pre image here with the post
# We need to do this so the classification inference pulls the images from the post 
# Since post is where the damage occurs
printf "Grabbing post image file for classification\n"
disaster_post_file="$input_post"

mkdir -p "$inference_base"/output_polygons

printf "Running classification\n" 

# Extracting polygons from post image I THINK ITS BROKEN HERE!!!!!! not generating output polygons?????
python3 ./process_data_inference.py --input_img "$disaster_post_file" --label_path "$label_temp"/"${input_image%.*}".json --output_dir "$inference_base"/output_polygons --output_csv "$inference_base"/output.csv >> "$LOGFILE" 2>&1

# Classifying extracted polygons
python3 ./damage_inference.py --test_data "$inference_base"/output_polygons --test_csv "$inference_base"/output.csv --model_weights "$classification_weights" --output_json /tmp/inference/classification_inference.json >> "$LOGFILE" 2>&1

printf "\n" >> "$LOGFILE"

# Combining the predicted polygons with the predicted labels, based off a UUID generated during the localization inference stage  
printf "Formatting json and scoring image\n"
python3 "$XBDIR"/utils/combine_jsons.py --polys "$label_temp"/"${input_image%.*}".json --classes /tmp/inference/classification_inference.json --output "$inference_base/inference.json" >> "$LOGFILE" 2>&1
printf "\n" >> "$LOGFILE"

# Transforming the inference json file to the image required for scoring
printf "Finalizing output file" 
python3 "$XBDIR"/utils/inference_image_output.py --input "$inference_base"/inference.json --output "$output_file"  >> "$LOGFILE" 2>&1

#Cleaning up by removing the temporary working directory we created
printf "Cleaning up\n"
rm -rf "$inference_base"

printf "==========\n" >> "$LOGFILE"
printf "Done!\n"